In [ ]:
import warnings; warnings.simplefilter('ignore')
import numpy as np
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
oe = preprocessing.OneHotEncoder(sparse=False)
import re
import matplotlib.pyplot as plt
%matplotlib inline
from pystruct.datasets import load_letters
import pycrfsuite

In [ ]:
letters = load_letters()
X, y, folds = letters['data'], letters['labels'], letters['folds']
# we convert the lists to object arrays, as that makes slicing much more
# convenient
X, y = np.array(X), np.array(y)

In [ ]:
X = X[:5000]
y = y[:5000]

In [ ]:
# Count how many letters are in the first 5,000 sequences
sum([len(x) for x in X[:5000]])

In [ ]:
print (X.shape, y.shape)

In [ ]:
print (X[0].shape)
print (X[150].shape)

In [ ]:
print (y[0].shape)
print (y[150].shape)

In [ ]:
def convert_letter_to_dict(letter):
    d = {}
    #print ("letter:", letter.shape)    # ('letter:', (128,))
    
    # You are welcome to choose another kind of features
    for ind, i in enumerate(letter):
        #print (d)
        d["p_"+str(ind)] = i
    return d

In [ ]:
def word2features(word, i):
    letter = word[i]
    # You are welcome to choose another kind of features
    features = {
    'letter': convert_letter_to_dict(letter)
    }
    
    # Comment Both "if"s --> Window size 1
    # Comment first "if" --> Window size 2
    # Don't Comment --> Window size 3
    if i > 0:
        # The letter before the current one
        letter = word[i-1]
        features.update({
        '-1:letter': str(convert_letter_to_dict(letter))
        })
    if i < len(word)-1:
        # The letter after the current one
        letter = word[i+1]
        features.update({
        '+1:letter': str(convert_letter_to_dict(letter))
        })
    return features

In [ ]:
def create_word_features(data):
     #print (data.shape)    # (9, 128)
     return [word2features(data, i) for i in range(len(data))]
X_features = [create_word_features(word) for word in X]

In [ ]:
X_features = np.array(X_features)

In [ ]:
X_features[0]

In [ ]:
trainer = pycrfsuite.Trainer(verbose=True)

In [ ]:
X_train, X_test = X_features[:1000], X_features[1000:]
y_train, y_test = y[:1000], y[1000:]

In [ ]:
X_train, X_test = X_features[:2500], X_features[2500:]
y_train, y_test = y[:2500], y[2500:]

In [ ]:
X_train, X_test = X_features[:4000], X_features[4000:]
y_train, y_test = y[:4000], y[4000:]

In [ ]:
print (X_train.shape, X_test.shape)

In [ ]:
print (y_train.shape, y_test.shape)

In [ ]:
y_tr = []
for y_i in y_train:
    z = []
    #print (y_i.shape)    # (9,)
    for i in y_i:
        z.append(str(i))
        #print ("str(i):", str(i), type(i))    # ('str(i):', '14', <type 'numpy.int64'>)
    y_tr.append(z)
    
y_te = []
for y_i in y_test:
    z = []
    for i in y_i:
        z.append(str(i))
    y_te.append(z)

In [ ]:
cnt=0
a=0
for xseq, yseq in zip(X_train, y_tr):
    cnt +=1
    ystr = yseq
    #print ("ystr:", ystr)    # ('ystr:', ['14', '12', '12', '0', '13', '3', '8', '13', '6'])
    if(len(ystr)!=len(xseq)):
       print(cnt)
       continue
    a+=1
    trainer.append(xseq, ystr)

In [ ]:
trainer.set_params({
    'c1': 1.0,   # coefficient for L1 penalty
    'c2': 1e-3,  # coefficient for L2 penalty
    'max_iterations': 1000,  # stop earlier

    # include transitions that are possible, but not observed
    'feature.possible_transitions': True
})

trainer.train('ocr.crfsuite')

In [ ]:
tagger = pycrfsuite.Tagger()
tagger.open('ocr.crfsuite')

In [ ]:
tagger.tag(X_test[100])

In [ ]:
y_te[100]

In [ ]:
def calc_acc(data, y):
    num = 0
    den = 0
    for i in range(len(data)):
        pred = tagger.tag(data[i])
        for j in range(len(pred)):
            num = num + int(pred[j] == y[i][j])
        den = den + len(pred)
    return num / den

In [ ]:
print("Train acc:", calc_acc(X_train, y_tr))
print("Test acc:", calc_acc(X_test, y_te))